# Project: Identify Customer Segments

In this project, you will apply unsupervised learning techniques to identify segments of the population that form the core customer base for a mail-order sales company in Germany. These segments can then be used to direct marketing campaigns towards audiences that will have the highest expected rate of returns. The data that you will use has been provided by our partners at Bertelsmann Arvato Analytics, and represents a real-life data science task.

This notebook will help you complete this task by providing a framework within which you will perform your analysis steps. In each step of the project, you will see some text describing the subtask that you will perform, followed by one or more code cells for you to complete your work. **Feel free to add additional code and markdown cells as you go along so that you can explore everything in precise chunks.** The code cells provided in the base template will outline only the major tasks, and will usually not be enough to cover all of the minor tasks that comprise it.

It should be noted that while there will be precise guidelines on how you should handle certain tasks in the project, there will also be places where an exact specification is not provided. **There will be times in the project where you will need to make and justify your own decisions on how to treat the data.** These are places where there may not be only one way to handle the data. In real-life tasks, there may be many valid ways to approach an analysis task. One of the most important things you can do is clearly document your approach so that other scientists can understand the decisions you've made.

At the end of most sections, there will be a Markdown cell labeled **Discussion**. In these cells, you will report your findings for the completed section, as well as document the decisions that you made in your approach to each subtask. **Your project will be evaluated not just on the code used to complete the tasks outlined, but also your communication about your observations and conclusions at each stage.**

In [ ]:
# import libraries here; add more as necessary
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import operator
import time
import warnings
warnings.filterwarnings('ignore')
import sklearn
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import re

# magic word for producing visualizations in notebook
%matplotlib inline

pd.options.display.max_rows = None
pd.options.display.max_columns = None

print('done')

### Step 0: Load the Data

There are four files associated with this project (not including this one):

- `Udacity_AZDIAS_Subset.csv`: Demographics data for the general population of Germany; 891211 persons (rows) x 85 features (columns).
- `Udacity_CUSTOMERS_Subset.csv`: Demographics data for customers of a mail-order company; 191652 persons (rows) x 85 features (columns).
- `Data_Dictionary.md`: Detailed information file about the features in the provided datasets.
- `AZDIAS_Feature_Summary.csv`: Summary of feature attributes for demographics data; 85 features (rows) x 4 columns

Each row of the demographics files represents a single person, but also includes information outside of individuals, including information about their household, building, and neighborhood. You will use this information to cluster the general population into groups with similar demographic properties. Then, you will see how the people in the customers dataset fit into those created clusters. The hope here is that certain clusters are over-represented in the customers data, as compared to the general population; those over-represented clusters will be assumed to be part of the core userbase. This information can then be used for further applications, such as targeting for a marketing campaign.

To start off with, load in the demographics data for the general population into a pandas DataFrame, and do the same for the feature attributes summary. Note for all of the `.csv` data files in this project: they're semicolon (`;`) delimited, so you'll need an additional argument in your [`read_csv()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) call to read in the data properly. Also, considering the size of the main dataset, it may take some time for it to load completely.

Once the dataset is loaded, it's recommended that you take a little bit of time just browsing the general structure of the dataset and feature summary file. You'll be getting deep into the innards of the cleaning in the first major step of the project, so gaining some general familiarity can help you get your bearings.

In [ ]:
# Load in the general demographics data.
azdias = pd.read_csv('Udacity_AZDIAS_Subset.csv' , delimiter=';')
azdias.shape

In [ ]:
# Load in the feature summary file.
feat_info = pd.read_csv('AZDIAS_Feature_Summary.csv' , delimiter=';')
feat_info.shape

In [ ]:
azdias.info()

In [ ]:
azdias.head()

In [ ]:
azdias.info()

In [ ]:
feat_info.head()

In [ ]:
display(feat_info['missing_or_unknown'].head(15))

In [ ]:
# Check the structure of the data after it's loaded (e.g. print the number of
# rows and columns, print the first few rows).
feat_info.head(15)


> **Tip**: Add additional cells to keep everything in reasonably-sized chunks! Keyboard shortcut `esc --> a` (press escape to enter command mode, then press the 'A' key) adds a new cell before the active cell, and `esc --> b` adds a new cell after the active cell. If you need to convert an active cell to a markdown cell, use `esc --> m` and to convert to a code cell, use `esc --> y`. 

## Step 1: Preprocessing

### Step 1.1: Assess Missing Data

The feature summary file contains a summary of properties for each demographics data column. You will use this file to help you make cleaning decisions during this stage of the project. First of all, you should assess the demographics data in terms of missing data. Pay attention to the following points as you perform your analysis, and take notes on what you observe. Make sure that you fill in the **Discussion** cell with your findings and decisions at the end of each step that has one!

#### Step 1.1.1: Convert Missing Value Codes to NaNs
The fourth column of the feature attributes summary (loaded in above as `feat_info`) documents the codes from the data dictionary that indicate missing or unknown data. While the file encodes this as a list (e.g. `[-1,0]`), this will get read in as a string object. You'll need to do a little bit of parsing to make use of it to identify and clean the data. Convert data that matches a 'missing' or 'unknown' value code into a numpy NaN value. You might want to see how much data takes on a 'missing' or 'unknown' code, and how much data is naturally missing, as a point of interest.

**As one more reminder, you are encouraged to add additional cells to break up your analysis into manageable chunks.**

In [ ]:
col_names = azdias.columns
col_names

In [ ]:
number_miss = azdias.isnull().sum()
display(number_miss.head(15))
print("Total Missing: ", number_miss.sum())

In [ ]:
for i in range(len(feat_info)):
     
    miss_data = re.sub('[\[|\]]', '',feat_info.iloc[i]['missing_or_unknown']).split(',')
    #check for string values
    if miss_data != ['']:
        miss_data = [np.int64(data) if (data!='X' and data!='XX') else data for data in miss_data]
        azdias = azdias.replace({feat_info.iloc[i]['attribute']: miss_data}, np.nan)         

In [ ]:
display(azdias.head())

#### Step 1.1.2: Assess Missing Data in Each Column

How much missing data is present in each column? There are a few columns that are outliers in terms of the proportion of values that are missing. You will want to use matplotlib's [`hist()`](https://matplotlib.org/api/_as_gen/matplotlib.pyplot.hist.html) function to visualize the distribution of missing value counts to find these columns. Identify and document these columns. While some of these columns might have justifications for keeping or re-encoding the data, for this project you should just remove them from the dataframe. (Feel free to make remarks about these outlier columns in the discussion, however!)

For the remaining features, are there any patterns in which columns have, or share, missing data?

In [ ]:
number_miss_cols = azdias.isnull().sum()
percent_miss_cols = number_miss_cols/len(azdias)*100
display(number_miss_cols.head(15))
print("Missing or unkown values after converting is", number_miss_cols.sum())


In [ ]:
azdias.isnull().sum().sum()

In [ ]:
# Perform an assessment of how much missing data there is in each column of the
# dataset.

missing_data = azdias.isnull().sum()
#percentage
missing_data = missing_data[missing_data > 0]/(azdias.shape[0]) * 100
#sort 
missing_data.sort_values(inplace=True)

missing_data


In [ ]:
plt.hist(missing_data, bins = 25)

plt.xlabel('Percentage of Missing Data(%)')
plt.ylabel('Counts')
plt.title('Missing Data Chart')
plt.grid(True)
plt.show()

In [ ]:
# Investigate patterns in the amount of missing data in each column.
missing_data.plot.barh(figsize=(15,30))
plt.title('Missing Values Chart')
plt.xlabel('Columns')
plt.ylabel('Percentages')

plt.show()


In [ ]:
#25% of data missing
missing_data = [col for col in azdias.columns 
               if (azdias[col].isnull().sum()/azdias.shape[0])
               * 100 > 25]

missing_data

In [ ]:
# Remove the outlier columns from the dataset. (You'll perform other data
# engineering tasks such as re-encoding and imputation later.)

#for col in missing_data:
#    azdias.drop(col, axis=1, inplace=True)



In [ ]:
#check again 25% of data missing
missing_data1 = [col for col in azdias.columns 
               if (azdias[col].isnull().sum()/azdias.shape[0])
               * 100 > 25]

missing_data1

In [ ]:
azdias.isnull().sum()

In [ ]:
columns = percent_miss_cols[percent_miss_cols > 20]
columns_names = columns.keys().tolist() 
print(columns, columns_names)
# display the dataset before removing outliers
#(azdias.head())

In [ ]:
percent_miss_cols = number_miss_cols/len(azdias)*100
display(percent_miss_cols.head())
print(percent_miss_cols.describe())

In [ ]:
azdias = azdias.drop(columns_names, axis=1)
display(azdias.head())

#### Discussion 1.1.2: Assess Missing Data in Each Column

Executing the code segments above and graphing the outliers for more than 20% I have removed the following columns since they had 20% data missing: ['AGER_TYP', 'GEBURTSJAHR', 'TITEL_KZ', 'ALTER_HH', 'KK_KUNDENTYP', 'KBA05_BAUMAX']. This step is important to ensure less bias data are in the training model

#### Step 1.1.3: Assess Missing Data in Each Row

Now, you'll perform a similar assessment for the rows of the dataset. How much data is missing in each row? As with the columns, you should see some groups of points that have a very different numbers of missing values. Divide the data into two subsets: one for data points that are above some threshold for missing values, and a second subset for points below that threshold.

In order to know what to do with the outlier rows, we should see if the distribution of data values on columns that are not missing data (or are missing very little data) are similar or different between the two groups. Select at least five of these columns and compare the distribution of values.
- You can use seaborn's [`countplot()`](https://seaborn.pydata.org/generated/seaborn.countplot.html) function to create a bar chart of code frequencies and matplotlib's [`subplot()`](https://matplotlib.org/api/_as_gen/matplotlib.pyplot.subplot.html) function to put bar charts for the two subplots side by side.
- To reduce repeated code, you might want to write a function that can perform this comparison, taking as one of its arguments a column to be compared.

Depending on what you observe in your comparison, this will have implications on how you approach your conclusions later in the analysis. If the distributions of non-missing features look similar between the data with many missing values and the data with few or no missing values, then we could argue that simply dropping those points from the analysis won't present a major issue. On the other hand, if the data with many missing values looks very different from the data with few or no missing values, then we should make a note on those data as special. We'll revisit these data later on. **Either way, you should continue your analysis for now using just the subset of the data with few or no missing values.**

In [ ]:
num_mis_rows = azdias.isnull().sum(axis = 1)
display(num_mis_rows.head())
print("Summary", num_mis_rows.sum())

In [ ]:
num_mis_cols = azdias.isnull().sum()
display(num_mis_cols.head())
print(num_mis_cols.describe())
print("Summary Missing Data", num_mis_cols.sum())

In [ ]:
percent_mis_cols = num_mis_cols/len(azdias)*100
display(percent_mis_cols.head())
print(percent_mis_cols.describe())

In [ ]:
percent_mis_rows = num_mis_rows/azdias.shape[1]*100
print(percent_mis_rows.describe())

In [ ]:
# How much data is missing in each row of the dataset?
mis_row_data = azdias.isnull().sum(axis=1)
mis_row_data = mis_row_data[mis_row_data > 0]/(len(azdias.columns)) * 100
mis_row_data.sort_values(inplace=True)

plt.hist(mis_row_data, bins=50, color='green')

plt.xlabel('Percentage')
plt.ylabel('Counts')
plt.title('Missing Totals')
plt.grid(True)
plt.show()


In [ ]:
mis_row_data.sum()

In [ ]:
# Write code to divide the data into two subsets based on the number of missing
# values in each row.
low = azdias[azdias.isnull().sum(axis=1) < 30]

high = azdias[azdias.isnull().sum(axis=1) >= 30]

print(f'High missing row value is: {len(high)/len(azdias)*100},low missing row value is: {len(low)/len(azdias)*100}')

In [ ]:
low.index, high.index

In [ ]:
display(low.head())

In [ ]:
display(high.head())

In [ ]:
percent_missing_rows1 = num_mis_rows/azdias.shape[1]*100
print(percent_missing_rows1.describe())

In [ ]:
def drawGraphs():
    plt.figure(figsize=(25,25))
    for i, col in enumerate(azdias.columns[:10]):
        plt.subplot(5, 2, i+1)
        sns.displot(low[col][low[col].notnull()], label='LOW')
        sns.displot(high[col][high[col].notnull()], label='HIGH')
        plt.title('Column: {}'.format(col))
        plt.legend();

# uncomment only when submission. creates a memory hog
drawGraphs()

In [ ]:
complete_cols = percent_mis_cols[percent_mis_cols == 0]
complete_cols = complete_cols[:8]
complete_cols_names = complete_cols.keys().tolist()
print(complete_cols, complete_cols_names)

In [ ]:
# Compare the distribution of values for at least five columns where there are
# no or few missing values, between the two subsets.

def drawMissing():
    
    col_names1 = low.columns
    fig, axs = plt.subplots(8,2, figsize=(20, 25))
    fig.subplots_adjust(hspace =2 , wspace=.2)
    axs = axs.ravel()

    for i in range(8):
        sns.countplot(low[col_names1[i]], ax=axs[i*2])
        axs[i*2].set_title('Missing Low')
        sns.countplot(high[col_names1[i]], ax=axs[i*2+1])
        axs[i*2+1].set_title('Missing High')
        
# uncomment only when submission. create a memory hog
#drawMissing()

In [ ]:
#Trying a different method since the above drawMissing takes too long
na_count=pd.DataFrame(azdias.shape[1] - azdias.count(axis=1))
na_count.columns=['na_count']
na_count1=na_count[na_count['na_count']<30]
na_count2=na_count[na_count['na_count']>=30]

azdias1=azdias.iloc[na_count1.index,:]
azdias2=azdias.iloc[na_count2.index,:]

In [ ]:
fig1 = plt.figure()
ax1 = fig1.add_subplot(211)
ax1.hist(azdias1['ANREDE_KZ'])
ax2 = fig1.add_subplot(212)
ax2.hist(azdias2['ANREDE_KZ'])

plt.show()

In [ ]:
fig1 = plt.figure()
ax1 = fig1.add_subplot(211)
ax1.hist(azdias1['FINANZ_MINIMALIST'])
ax2 = fig1.add_subplot(212)
ax2.hist(azdias2['FINANZ_MINIMALIST'])

plt.show()

In [ ]:
fig1 = plt.figure()
ax1 = fig1.add_subplot(211)
ax1.hist(azdias1['GREEN_AVANTGARDE'])
ax2 = fig1.add_subplot(212)
ax2.hist(azdias2['GREEN_AVANTGARDE'])

plt.show()

In [ ]:
fig1 = plt.figure()
ax1 = fig1.add_subplot(211)
ax1.hist(azdias1['SEMIO_SOZ'])
ax2 = fig1.add_subplot(212)
ax2.hist(azdias2['SEMIO_SOZ'])

plt.show()

In [ ]:
fig1 = plt.figure()
ax1 = fig1.add_subplot(211)
ax1.hist(azdias1['ZABEOTYP'])
ax2 = fig1.add_subplot(212)
ax2.hist(azdias2['ZABEOTYP'])

plt.show()

#### Discussion 1.1.3: Assess Missing Data in Each Row

In many instances, datasets that contain a substantial amount of missing values are markedly different from those that are complete or have very few missing values. This discrepancy often arises because the missing values can introduce biases or obscure important patterns within the data. As a result, relying on datasets with numerous missing values can lead to flawed analyses and incorrect predictions. To ensure the accuracy and reliability of predictive models, it is generally advisable to use datasets that have minimal or no missing values. By doing so, the integrity of the data is preserved, and the models built from such data are more likely to produce valid and actionable insights.

### Step 1.2: Select and Re-Encode Features

Checking for missing data isn't the only way in which you can prepare a dataset for analysis. Since the unsupervised learning techniques to be used will only work on data that is encoded numerically, you need to make a few encoding changes or additional assumptions to be able to make progress. In addition, while almost all of the values in the dataset are encoded using numbers, not all of them represent numeric values. Check the third column of the feature summary (`feat_info`) for a summary of types of measurement.
- For numeric and interval data, these features can be kept without changes.
- Most of the variables in the dataset are ordinal in nature. While ordinal values may technically be non-linear in spacing, make the simplifying assumption that the ordinal variables can be treated as being interval in nature (that is, kept without any changes).
- Special handling may be necessary for the remaining two variable types: categorical, and 'mixed'.

In the first two parts of this sub-step, you will perform an investigation of the categorical and mixed-type features and make a decision on each of them, whether you will keep, drop, or re-encode each. Then, in the last part, you will create a new data frame with only the selected and engineered columns.

Data wrangling is often the trickiest part of the data analysis process, and there's a lot of it to be done here. But stick with it: once you're done with this step, you'll be ready to get to the machine learning parts of the project!

In [ ]:
# How many features are there of each data type?

#feat_info.set_index('attribute', inplace=True)
#feat_info.head(n=10)



In [ ]:
features = azdias.keys().tolist()
features = feat_info[feat_info['attribute'].isin(features)]  
features['type'].value_counts()

In [ ]:
#feat_info['col_names'] = feat_info.index
#feat_info.head()

#### Step 1.2.1: Re-Encode Categorical Features

For categorical data, you would ordinarily need to encode the levels as dummy variables. Depending on the number of categories, perform one of the following:
- For binary (two-level) categoricals that take numeric values, you can keep them without needing to do anything.
- There is one binary variable that takes on non-numeric values. For this one, you need to re-encode the values as numbers or create a dummy variable.
- For multi-level categoricals (three or more values), you can choose to encode the values using multiple dummy variables (e.g. via [OneHotEncoder](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html)), or (to keep things straightforward) just drop them from the analysis. As always, document your choices in the Discussion section.

In [ ]:
# Assess categorical variables: which are binary, which are multi-level, and
# which one needs to be re-encoded?
categorical_features = features[features['type'] == 'categorical']['attribute']
mixed_features = features[features['type'] == 'mixed']['attribute']

display(features.head())
display(categorical_features.head())
display(mixed_features.head())

In [ ]:
cat_columns = feat_info.loc[feat_info['type'] == 'categorical'].values
cat_columns

In [ ]:
# Split the categorical features 
col_binary = []
col_multi = []

for col in categorical_features:
    if (low[col].nunique() > 2):
        col_multi.append(col)
    else:
        col_binary.append(col)
        
display(col_binary)
display(col_multi)

In [ ]:
low[col_binary].head()

In [ ]:
low.loc[:, 'OST_WEST_KZ'] = low['OST_WEST_KZ'].replace({'W': 0, 'O': 1})
low[col_binary].head()

In [ ]:
low[col_multi].head()

In [ ]:
low = low.drop(col_multi, axis = 1)
display(low.head())

#### Discussion 1.2.1: Re-Encode Categorical Features

I retained all numerical, ordinal, and mixed types of data to ensure a comprehensive analysis. However, I excluded multi-categorical variables, as they can complicate the modeling process, and focused on retaining only binary categorical variables for simplicity and clarity. Additionally, I standardized the binary categorical feature OST_WEST_KZ by converting its values from W and O to 0 and 1.  This adjustment enhances the consistency and interpretability of the data, facilitating more accurate analysis and modeling.

#### Step 1.2.2: Engineer Mixed-Type Features

There are a handful of features that are marked as "mixed" in the feature summary that require special treatment in order to be included in the analysis. There are two in particular that deserve attention; the handling of the rest are up to your own choices:
- "PRAEGENDE_JUGENDJAHRE" combines information on three dimensions: generation by decade, movement (mainstream vs. avantgarde), and nation (east vs. west). While there aren't enough levels to disentangle east from west, you should create two new variables to capture the other two dimensions: an interval-type variable for decade, and a binary variable for movement.
- "CAMEO_INTL_2015" combines information on two axes: wealth and life stage. Break up the two-digit codes by their 'tens'-place and 'ones'-place digits into two new ordinal variables (which, for the purposes of this project, is equivalent to just treating them as their raw numeric values).
- If you decide to keep or engineer new features around the other mixed-type features, make sure you note your steps in the Discussion section.

Be sure to check `Data_Dictionary.md` for the details needed to finish these tasks.

In [ ]:
# Investigate "PRAEGENDE_JUGENDJAHRE" and engineer two new variables.

display(azdias[['PRAEGENDE_JUGENDJAHRE']].head())

In [ ]:
display(low['PRAEGENDE_JUGENDJAHRE'].head())

In [ ]:
def a_interval(x):
    if x in (1,2):
        return 1
    elif x in (3,4):
        return 2
    elif x in (5,6,7):
        return 3
    elif x in (8,9):
        return 4
    elif x in (10,11,12,13):
        return 5
    elif x in (14,15):
        return 6
    
def b_movement(x):
    if x in (2,4,6,7,9,11,13,15):
        return 0
    elif x in (1,3,5,8,10,12,14):
        return 1

In [ ]:
low['DECADES'] = low['PRAEGENDE_JUGENDJAHRE'].apply(a_interval)
low['MOVEMENTS'] = low['PRAEGENDE_JUGENDJAHRE'].apply(b_movement)

In [ ]:
display(low['DECADES'].head())

In [ ]:
display(low['MOVEMENTS'].head())

In [ ]:
# Investigate "CAMEO_INTL_2015" and engineer two new variables.
low[['CAMEO_INTL_2015']].head()


In [ ]:
def a_wealth(x):
    if x // 10 ==1:
        return 1
    if x // 10 ==2:
        return 2
    if x // 10 ==3:
        return 3
    if x // 10 ==4:
        return 4
    if x // 10 ==5:
        return 5

# Adding a feature based on lfe stage
def b_life_stage(x):
    if x % 10 ==1:
        return 1
    if x % 10 ==2:
        return 2
    if x % 10 ==3:
        return 3
    if x % 10 ==4:
        return 4
    if x % 10 ==5:
        return 5

In [ ]:
low['CAMEO_INTL_2015'] = pd.to_numeric(azdias['CAMEO_INTL_2015'])
low['WEALTH'] = low['CAMEO_INTL_2015'].apply(a_wealth)
low['LIFE_STAGE'] = low['CAMEO_INTL_2015'].apply(b_life_stage)


In [ ]:
display(low.head())

In [ ]:
low = low.drop(mixed_features, axis = 1)

In [ ]:
display(low.head())

#### Discussion 1.2.2: Engineer Mixed-Type Features

I included two new columns that required encoding to better represent categorical data numerically, which enhances the compatibility of the dataset with various analytical and machine learning techniques. Furthermore, I updated the values in the DECADES and MOVEMENTS columns based on a predefined function. This function systematically transformed the data to ensure consistency and accuracy, reflecting the appropriate historical periods and artistic movements relevant to the dataset. By making these adjustments, I improved the dataset's overall quality and usability for subsequent analysis and modeling tasks.

#### Step 1.2.3: Complete Feature Selection

In order to finish this step up, you need to make sure that your data frame now only has the columns that you want to keep. To summarize, the dataframe should consist of the following:
- All numeric, interval, and ordinal type columns from the original dataset.
- Binary categorical features (all numerically-encoded).
- Engineered features from other multi-level categorical features and mixed features.

Make sure that for any new columns that you have engineered, that you've excluded the original columns from the final dataset. Otherwise, their values will interfere with the analysis later on the project. For example, you should not keep "PRAEGENDE_JUGENDJAHRE", since its values won't be useful for the algorithm: only the values derived from it in the engineered features you created should be retained. As a reminder, your data should only be from **the subset with few or no missing values**.

In [ ]:
# If there are other re-engineering tasks you need to perform, make sure you
# take care of them here. (Dealing with missing data will come in step 2.1.)
np.unique(azdias.dtypes.values)


In [ ]:
low.dtypes

In [ ]:
# Do whatever you need to in order to ensure that the dataframe only contains
# the columns that should be passed to the algorithm functions.
low_miss_col = low.keys().tolist()
imp_mean = SimpleImputer(missing_values = np.nan, strategy = 'most_frequent')
low = imp_mean.fit_transform(low)
low = pd.DataFrame(low, columns=low_miss_col)

low.head(40)

### Step 1.3: Create a Cleaning Function

Even though you've finished cleaning up the general population demographics data, it's important to look ahead to the future and realize that you'll need to perform the same cleaning steps on the customer demographics data. In this substep, complete the function below to execute the main feature selection, encoding, and re-engineering steps you performed above. Then, when it comes to looking at the customer data in Step 3, you can just run this function on that DataFrame to get the trimmed dataset in a single step.

In [ ]:
display(feat_info.head())

In [ ]:
def clean_data(df):
    """
    Perform feature trimming, re-encoding, and engineering for demographics
    data
    
    INPUT: Demographics DataFrame
    OUTPUT: Trimmed and cleaned demographics DataFrame
    """
    
    dec = {1:1, 2:1, 3:2, 4:2, 5:3, 6:3, 7:3, 8:4, 9:4, 10:5, 11:5, 12:5, 13:5, 14:6, 15:6}
    mov = {1:0, 2:1, 3:0, 4:1, 5:0, 6:1, 7:1, 8:0, 9:1, 10:0, 11:1, 12:0, 13:1, 14:0, 15:1}


    display(df.head())
    
    # Put in code here to execute all main cleaning steps:
    # convert missing value codes into NaNs, ...
    for i in range(len(feat_info)):
        missing_data = re.sub('[\[|\]]', '', feat_info.iloc[i]['missing_or_unknown']).split(',')
        if missing_data != ['']:
            missing_data = [int(data) if (data != 'X' and data != 'XX') else data for data in missing_data]
            df = df.replace({feat_info.iloc[i]['attribute']: missing_data}, np.nan)
            
    print("Missing Values Conversions")
    display(df.head())
    
    # remove selected columns and rows, ...
    df = df.drop(columns_names, axis=1)
    df = df[df.isnull().sum(axis=1) < 30]
    
    print("Removal of Selected Rows And Columns")
    display(df.head())
    
    
    # 1) convert the binary categorical feature to numerical values
    df.replace({'OST_WEST_KZ':{'W':0, 'O': 1}}, inplace = True)
    
    # 2) drop the multi categorical features
    df = df.drop(col_multi, axis = 1)

    welth = {
        '11':1, '12':1, '13':1, '14':1, '15':1, 
        '21':2, '22':2, '23':2, '24':2, '25':2, 
        '31':3, '32':3, '33':3, '34':3, '35':3, 
        '41':4, '42':4, '43':4, '44':4, '45':4, 
        '51':5, '52':5, '53':5, '54':5, '55':5, 
    }

    life ={
        '11':1, '12':2, '13':3, '14':4, '15':5, 
        '21':1, '22':2, '23':3, '24':4, '25':5, 
        '31':1, '32':2, '33':3, '34':4, '35':5, 
        '41':1, '42':2, '43':3, '44':4, '45':5, 
        '51':1, '52':2, '53':3, '54':4, '55':5, 
    }
    
    # 3) re-encode  
    df['DECADE'] = df['PRAEGENDE_JUGENDJAHRE']
    df['DECADE'].replace(dec, inplace = True)
    df['MOVEMENT'] = df['PRAEGENDE_JUGENDJAHRE']
    df['MOVEMENT'].replace(mov, inplace = True)
    
    df['WEALTH'] = df['CAMEO_INTL_2015']
    df['WEALTH'].replace(welth, inplace = True)
    df['LIFE_STAGE'] = df['CAMEO_INTL_2015']
    df['LIFE_STAGE'].replace(life, inplace = True)
    
    # 4) drop the rest of the mixed features
    df = df.drop(mixed_features, axis = 1)
    
    print("Re-encoding And Feature Engineering")
    display(df.head())

    columns_removed1 = ['AGER_TYP', 'GEBURTSJAHR', 'TITEL_KZ', 
                       'ALTER_HH', 'KK_KUNDENTYP', 'KBA05_BAUMAX',
                       'CAMEO_DEU_2015']
    # replacing the NaNs
    df_cols = df.keys().tolist()
    imp_mean = SimpleImputer(missing_values = np.nan, strategy = 'most_frequent')
    df = imp_mean.fit_transform(df)
    df = pd.DataFrame(df, columns=df_cols)
    
    print("Replacing NaN Values")
    display(df.head())
    # Return the cleaned dataframe.
    return df

In [ ]:
print('234')

## Step 2: Feature Transformation

### Step 2.1: Apply Feature Scaling

Before we apply dimensionality reduction techniques to the data, we need to perform feature scaling so that the principal component vectors are not influenced by the natural differences in scale for features. Starting from this part of the project, you'll want to keep an eye on the [API reference page for sklearn](http://scikit-learn.org/stable/modules/classes.html) to help you navigate to all of the classes and functions that you'll need. In this substep, you'll need to check the following:

- sklearn requires that data not have missing values in order for its estimators to work properly. So, before applying the scaler to your data, make sure that you've cleaned the DataFrame of the remaining missing values. This can be as simple as just removing all data points with missing data, or applying an [SimpleImputer](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html) to replace all missing values. You might also try a more complicated procedure where you temporarily remove missing values in order to compute the scaling parameters before re-introducing those missing values and applying imputation. Think about how much missing data you have and what possible effects each approach might have on your analysis, and justify your decision in the discussion section below.
- For the actual scaling function, a [StandardScaler](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) instance is suggested, scaling each feature to mean 0 and standard deviation 1.
- For these classes, you can make use of the `.fit_transform()` method to both fit a procedure to the data as well as apply the transformation to the data at the same time. Don't forget to keep the fit sklearn objects handy, since you'll be applying them to the customer demographics data towards the end of the project.

In [ ]:
azdias_g = pd.read_csv('Udacity_AZDIAS_Subset.csv', delimiter = ';')
azdias_g = clean_data(azdias_g)
azdias_g_col = azdias_g.keys().tolist()
print('done azdias_g')

In [ ]:
scaler = StandardScaler()
azdias_g = scaler.fit_transform(azdias_g)

### Discussion 2.1: Apply Feature Scaling

I processed the azdias dataset through a clean data function to ensure the data's reliability and suitability for future training purposes. This function systematically addressed issues such as missing values, outliers, and inconsistencies, resulting in a more refined and trustworthy dataset. Additionally, I applied the scaler.fit_transformation method to scale the dataset. This step normalized the data, ensuring that all features are on a similar scale, which is crucial for improving the performance and accuracy of machine learning models. By cleaning and scaling the data, I enhanced its quality and prepared it effectively for subsequent analytical and predictive tasks.

### Step 2.2: Perform Dimensionality Reduction

On your scaled data, you are now ready to apply dimensionality reduction techniques.

- Use sklearn's [PCA](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) class to apply principal component analysis on the data, thus finding the vectors of maximal variance in the data. To start, you should not set any parameters (so all components are computed) or set a number of components that is at least half the number of features (so there's enough features to see the general trend in variability).
- Check out the ratio of variance explained by each principal component as well as the cumulative variance explained. Try plotting the cumulative or sequential values using matplotlib's [`plot()`](https://matplotlib.org/api/_as_gen/matplotlib.pyplot.plot.html) function. Based on what you find, select a value for the number of transformed features you'll retain for the clustering part of the project.
- Once you've made a choice for the number of components to keep, make sure you re-fit a PCA instance to perform the decided-on transformation.

In [ ]:
# Apply PCA to the data.

pca = PCA(whiten= True)
pca_t1 = PCA()
pca_t1.fit(azdias_g)

In [ ]:
 #verify
print(pca_t1.explained_variance_ratio_)


In [ ]:
# Re-apply PCA to the data 
pca = PCA(n_components = 30)
az_data_pca = pca.fit_transform(azdias_g)


In [ ]:
print(az_data_pca.shape)

### Discussion 2.2: Perform Dimensionality Reduction

Based on the examples and best practices I reviewed for this project, I decided to reapply the Principal Component Analysis (PCA) method to the dataset. I set the number of components to 30, aiming to capture and retain at least 85% of the dataset's variance. By doing so, I intended to reduce the dimensionality of the dataset while preserving the most critical information. This approach helps in simplifying the data structure, enhancing computational efficiency, and improving the performance of subsequent machine learning models. The goal was to achieve a balance between reducing complexity and maintaining sufficient data to ensure robust and accurate predictions.

### Step 2.3: Interpret Principal Components

Now that we have our transformed principal components, it's a nice idea to check out the weight of each variable on the first few components to see if they can be interpreted in some fashion.

As a reminder, each principal component is a unit vector that points in the direction of highest variance (after accounting for the variance captured by earlier principal components). The further a weight is from zero, the more the principal component is in the direction of the corresponding feature. If two features have large weights of the same sign (both positive or both negative), then increases in one tend expect to be associated with increases in the other. To contrast, features with different signs can be expected to show a negative correlation: increases in one variable should result in a decrease in the other.

- To investigate the features, you should map each weight to their corresponding feature name, then sort the features according to weight. The most interesting features for each principal component, then, will be those at the beginning and end of the sorted list. Use the data dictionary document to help you understand these most prominent features, their relationships, and what a positive or negative value on the principal component might indicate.
- You should investigate and interpret feature associations from the first three principal components in this substep. To help facilitate this, you should write a function that you can call at any time to print the sorted list of feature weights, for the *i*-th principal component. This might come in handy in the next step of the project, when you interpret the tendencies of the discovered clusters.

In [ ]:
# Map weights 

def test_map_weight(pca, i):
    my_data = pd.DataFrame(pca.components_, columns = azdias_g_col)
    return my_data.iloc[i].sort_values(ascending = False)
    

test_map_weight(pca , 0)

In [ ]:
# second principal 
test_map_weight(pca , 1)


In [ ]:
#  third principal  

test_map_weight(pca , 2)

### Discussion 2.3: Interpret Principal Components

I did 3 test run with various level of component. As a comparison here are the results for the 3 component tests for a selected column:
PLZ8_ANTG3               0.225345
PLZ8_ANTG3               0.098404
PLZ8_ANTG3               0.048410

They all have a position correlation however, there is a high variation between them

## Step 3: Clustering

### Step 3.1: Apply Clustering to General Population

You've assessed and cleaned the demographics data, then scaled and transformed them. Now, it's time to see how the data clusters in the principal components space. In this substep, you will apply k-means clustering to the dataset and use the average within-cluster distances from each point to their assigned cluster's centroid to decide on a number of clusters to keep.

- Use sklearn's [KMeans](http://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html#sklearn.cluster.KMeans) class to perform k-means clustering on the PCA-transformed data.
- Then, compute the average difference from each point to its assigned cluster's center. **Hint**: The KMeans object's `.score()` method might be useful here, but note that in sklearn, scores tend to be defined so that larger is better. Try applying it to a small, toy dataset, or use an internet search to help your understanding.
- Perform the above two steps for a number of different cluster counts. You can then see how the average distance decreases with an increasing number of clusters. However, each additional cluster provides a smaller net benefit. Use this fact to select a final number of clusters in which to group the data. **Warning**: because of the large size of the dataset, it can take a long time for the algorithm to resolve. The more clusters to fit, the longer the algorithm will take. You should test for cluster counts through at least 10 clusters to get the full picture, but you shouldn't need to test for a number of clusters above about 30.
#- Once you've selected a final number of clusters to use, re-fit a KMeans instance to perform the clustering operation. Make sure that you also obtain the cluster assignments for the general demographics data, since you'll be using them in the final Step 3.3.

In [ ]:
# run kmeans and cacl distance
cluster_arr = [2, 4, 6, 8, 10, 12, 14, 16, 18, 20]
score_arr = []
for c in cluster_arr:
    kmeans = KMeans(n_clusters = c)
    model = kmeans.fit(az_data_pca)
    score1 = np.abs(model.score(az_data_pca))
    score_arr.append(score1)
    
print (score_arr)

In [ ]:
# Investigate the change in within-cluster distance across number of clusters.
# HINT: Use matplotlib's plot function to visualize this relationship.

plt.plot(cluster_arr, score_arr, marker = 'o', color = 'red')
plt.xlabel('Clusters')
plt.title("Clusters VS Score")
plt.ylabel('Score')
plt.show()

In [ ]:
# create model my_predict_az_data

kmeans = KMeans(n_clusters = 15)
model = kmeans.fit(az_data_pca)
my_predict_az_data = model.predict(az_data_pca)

### Discussion 3.1: Apply Clustering to General Population

The easiest way to accomplish this was to use the graph as shown above comparing the number of clusters vs score

### Step 3.2: Apply All Steps to the Customer Data

Now that you have clusters and cluster centers for the general population, it's time to see how the customer data maps on to those clusters. Take care to not confuse this for re-fitting all of the models to the customer data. Instead, you're going to use the fits from the general population to clean, transform, and cluster the customer data. In the last step of the project, you will interpret how the general population fits apply to the customer data.

- Don't forget when loading in the customers data, that it is semicolon (`;`) delimited.
- Apply the same feature wrangling, selection, and engineering steps to the customer demographics using the `clean_data()` function you created earlier. (You can assume that the customer demographics data has similar meaning behind missing data patterns as the general demographics data.)
- Use the sklearn objects from the general demographics data, and apply their transformations to the customers data. That is, you should not be using a `.fit()` or `.fit_transform()` method to re-fit the old objects, nor should you be creating new sklearn objects! Carry the data through the feature scaling, PCA, and clustering steps, obtaining cluster assignments for all of the data in the customer demographics data.

In [ ]:
# Load in the customer demographics data.
customers = pd.read_csv('Udacity_CUSTOMERS_Subset.csv', delimiter = ';')
display(customers.head())

In [ ]:
# clean data
customers = clean_data(customers)


### Step 3.3: Compare Customer Data to Demographics Data

At this point, you have clustered data based on demographics of the general population of Germany, and seen how the customer data for a mail-order sales company maps onto those demographic clusters. In this final substep, you will compare the two cluster distributions to see where the strongest customer base for the company is.

Consider the proportion of persons in each cluster for the general population, and the proportions for the customers. If we think the company's customer base to be universal, then the cluster assignment proportions should be fairly similar between the two. If there are only particular segments of the population that are interested in the company's products, then we should see a mismatch from one to the other. If there is a higher proportion of persons in a cluster for the customer data compared to the general population (e.g. 5% of persons are assigned to a cluster for the general population, but 15% of the customer data is closest to that cluster's centroid) then that suggests the people in that cluster to be a target audience for the company. On the other hand, the proportion of the data in a cluster being larger in the general population than the customer data (e.g. only 2% of customers closest to a population centroid that captures 6% of the data) suggests that group of persons to be outside of the target demographics.

Take a look at the following points in this step:

- Compute the proportion of data points in each cluster for the general population and the customer data. Visualizations will be useful here: both for the individual dataset proportions, but also to visualize the ratios in cluster representation between groups. Seaborn's [`countplot()`](https://seaborn.pydata.org/generated/seaborn.countplot.html) or [`barplot()`](https://seaborn.pydata.org/generated/seaborn.barplot.html) function could be handy.
  - Recall the analysis you performed in step 1.1.3 of the project, where you separated out certain data points from the dataset if they had more than a specified threshold of missing values. If you found that this group was qualitatively different from the main bulk of the data, you should treat this as an additional data cluster in this analysis. Make sure that you account for the number of data points in this subset, for both the general population and customer datasets, when making your computations!
- Which cluster or clusters are overrepresented in the customer dataset compared to the general population? Select at least one such cluster and infer what kind of people might be represented by that cluster. Use the principal component interpretations from step 2.3 or look at additional components to help you make this inference. Alternatively, you can use the `.inverse_transform()` method of the PCA and StandardScaler objects to transform centroids back to the original data space and interpret the retrieved values directly.
- Perform a similar investigation for the underrepresented clusters. Which cluster or clusters are underrepresented in the customer dataset compared to the general population, and what kinds of people are typified by these clusters?

In [ ]:
customers_list = scaler.transform(customers)
customers_pca = pca.transform(customers_list)
my_predict_customers = model.predict(customers_pca)

print('done my_predict_customers')

In [ ]:
# Compare the data for each list 
fig_sz = plt.figure(figsize=(14,4))
ax_1 = fig_sz.add_subplot(121)
ax_1.title.set_text('Customer Data')

print('DEBUG: ax_1')

sns.countplot(my_predict_customers, palette = 'Paired')

ax_2 = fig_sz.add_subplot(122)
ax_2.title.set_text('Demographic')

print('DEBUG: ax_2')

sns.countplot(my_predict_az_data, palette = 'Paired')
fig_sz.tight_layout() 
plt.show()


In [ ]:
# Review Populuation cluster 14?
clst_1 = model.cluster_centers_[14]
pca_1 = pca.inverse_transform(clst_1)
o_presented = scaler.inverse_transform(pca_1)

o_presented = pd.Series(data = o_presented, index = azdias_g_col)
display(o_presented.head(45))


In [ ]:
# Review Populuation cluster 8?
clst_2 = model.cluster_centers_[8]
pca_2 = pca.inverse_transform(clst_2)
u_presented = scaler.inverse_transform(pca_2)

u_presented = pd.Series(data = u_presented, index = azdias_g_col)
display(u_presented.head(45))


### Discussion 3.3: Compare Customer Data to Demographics Data

I encountered issues while running the aforementioned three code blocks in my Workspace, frequently receiving a "Kernel Dead" error. This persistent problem hindered the execution and testing of the code. Despite these technical difficulties, I formulated predictions based on the code and a comparative analysis of clusters 14 and 8. I anticipated an overrepresentation in one of the clusters, with cluster 8 expected to show an underrepresentation. These predictions were guided by the data's structure and previous insights, suggesting a significant disparity between these clusters. Moving forward, resolving the kernel error is crucial to validate these predictions and ensure the accuracy of the clustering analysis.

> Congratulations on making it this far in the project! Before you finish, make sure to check through the entire notebook from top to bottom to make sure that your analysis follows a logical flow and all of your findings are documented in **Discussion** cells. Once you've checked over all of your work, you should export the notebook as an HTML document to submit for evaluation. You can do this from the menu, navigating to **File -> Download as -> HTML (.html)**. You will submit both that document and this notebook for your project submission.